Zoe Erpelding, CPSC 322, Fall 2023, Notebook for HW-5

# 1. Load libraries and datasets

 Import the data table and utility functions.

In [99]:
from data_table import *
from data_learn import *
from data_eval import *
from data_util import *
from math import floor

Load and clean auto data.

In [100]:
auto = DataTable(['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
auto.load('auto-mpg.txt')

auto = remove_duplicates(auto)
auto = remove_missing(auto, ['mpg','cyls','disp','hp','weight','accl','year','origin','name'])


# 2. Exploring k-NN

*Note*: For the following, you must use functions we've already defined as appropriate

*TODO: Complete the following steps (you should read through the following before getting started to make sure you have a sense for what is being asked):*
1. Discretize the mpg values in the auto table using three equal-width bins
2. Normalize all of the columns except for model and origin
3. Create a train and test set using holdout with approximately half of the rows in the test set
4. Run knn over the train and test set to predict mpg class labels, using majority voting, k=5, the numerical columns cylinders, weight, and acceleration, and no nominal attributes. 
5. Print the resulting confusion matrix
6. Calculate and print the (average) accuracy across mpg labels, and the macro average f-measure. 
7. Run steps 1-6 3 times (i.e., put 1-6 in a for loop that iterates three times).
8. Redo 7 (as another for loop) that uses weighted voting instead.
9. Compare the performance differences, if any, between the results from 6 and 7.
10. Pick two other bin sizes (either equal width or hand-crafted cut-points) for mpg values and redo 6 (as another for loop) for each. Compare the results. 
11. Pick a bin size and voting method from above, and redo 6 but with two different k values (i.e., add two more for loops).

7. Steps 1-6 in a for loop

In [101]:
for i in range(3):
    # step 0
    auto = DataTable(['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
    auto.load('auto-mpg.txt')

    auto = remove_duplicates(auto)
    auto = remove_missing(auto, ['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
    
    # step 1
    width = (summary_stat(auto, 'mpg', max) - summary_stat(auto, 'mpg', min))/ 3
    discretize(auto, 'mpg', [summary_stat(auto, 'mpg', min) + width, summary_stat(auto, 'mpg', min) + width + width])

    # step 2
    normalize(auto, 'cyls')
    normalize(auto, 'disp')
    normalize(auto, 'hp')
    normalize(auto, 'weight')
    normalize(auto, 'accl')
    normalize(auto, 'year')

    # step 3
    total_rows = auto.row_count()
    test_size = floor(total_rows / 2)
    train, test = holdout(auto, test_size)

    # step 4
    matrix = knn_eval(train, test, majority_vote, 5, 'mpg', ['cyls', 'weight', 'accl'], [])
    print(matrix)

    # step 5
    avg_acc = (accuracy(matrix, 1) + accuracy(matrix, 2) + accuracy(matrix, 3)) / 3
    print(f' average accuracy is {avg_acc}')

    # step 6
    f = []
    for i in range(3):
        num = 0
        den = 0
        if i+1 in matrix.columns():
            num = (2*recall(matrix, i+1)*precision(matrix, i+1))
            den = (recall(matrix, i+1) + precision(matrix, i+1))
        else:
            fi = 0
        if den != 0:
            fi = num/den
        else:
            fi = 0
        f.append(fi)

    avg_f = (f[0]+f[1]+f[2])/3
    print(f'average f-measure is {avg_f}')


  actual    1    2    3
--------  ---  ---  ---
       1   90    0    0
       2   53    0    0
       3   10    0    0
 average accuracy is 0.7254901960784315
average f-measure is 0.24691358024691357
  actual    1    2    3
--------  ---  ---  ---
       1   77    3    0
       2   57    6    0
       3    9    1    0
 average accuracy is 0.6949891067538125
average f-measure is 0.2849888404283637
  actual    1    2    3
--------  ---  ---  ---
       1   78    0    0
       2   67    0    0
       3    8    0    0
 average accuracy is 0.6732026143790849
average f-measure is 0.22510822510822512


8. Another for loop but with weighted vote

In [102]:
for i in range(3):
    # step 0
    auto = DataTable(['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
    auto.load('auto-mpg.txt')

    auto = remove_duplicates(auto)
    auto = remove_missing(auto, ['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
    
    # step 1
    width = (summary_stat(auto, 'mpg', max) - summary_stat(auto, 'mpg', min))/ 3
    discretize(auto, 'mpg', [summary_stat(auto, 'mpg', min) + width, summary_stat(auto, 'mpg', min) + width + width])

    # step 2
    normalize(auto, 'cyls')
    normalize(auto, 'disp')
    normalize(auto, 'hp')
    normalize(auto, 'weight')
    normalize(auto, 'accl')
    normalize(auto, 'year')

    # step 3
    total_rows = auto.row_count()
    test_size = floor(total_rows / 2)
    train, test = holdout(auto, test_size)

    # step 4
    matrix = knn_eval(train, test, weighted_vote, 5, 'mpg', ['cyls', 'weight', 'accl'], [])
    print(matrix)

    # step 5
    avg_acc = (accuracy(matrix, 1) + accuracy(matrix, 2) + accuracy(matrix, 3)) / 3
    print(f' average accuracy is {avg_acc}')

    # step 6
    f = []
    for i in range(3):
        num = 0
        den = 0
        if i+1 in matrix.columns():
            num = (2*recall(matrix, i+1)*precision(matrix, i+1))
            den = (recall(matrix, i+1) + precision(matrix, i+1))
        else:
            fi = 0
        if den != 0:
            fi = num/den
        else:
            fi = 0
        f.append(fi)

    avg_f = (f[0]+f[1]+f[2])/3
    print(f'average f-measure is {avg_f}')

  actual    1    2    3
--------  ---  ---  ---
       1    1    8   73
       2    0    2   58
       3    0    2    9
 average accuracy is 0.3856209150326797
average f-measure is 0.06628574637032308
  actual    1    2    3
--------  ---  ---  ---
       1    1    0   81
       2    1    0   60
       3    0    0   10
 average accuracy is 0.38126361655773416
average f-measure is 0.049344375431331966
  actual    1    2    3
--------  ---  ---  ---
       1    3    0   75
       2    0    0   63
       3    0    0   12
 average accuracy is 0.3986928104575164
average f-measure is 0.07407407407407407


9. Compare the performance differences, if any, between the results from 6 and 7.

The majority vote seems to have a better accuracy and f-measure.

10. Pick two other bin sizes (either equal width or hand-crafted cut-points) for mpg values and redo 6 (as another for loop) for each. Compare the results. 


Choice 1 of bin size: [< 17, 17-24, < 24]

In [103]:
for i in range(3):
    # step 0
    auto = DataTable(['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
    auto.load('auto-mpg.txt')

    auto = remove_duplicates(auto)
    auto = remove_missing(auto, ['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
    
    # step 1
    discretize(auto, 'mpg', [17.0, 24.0])

    # step 2
    normalize(auto, 'cyls')
    normalize(auto, 'disp')
    normalize(auto, 'hp')
    normalize(auto, 'weight')
    normalize(auto, 'accl')
    normalize(auto, 'year')

    # step 3
    total_rows = auto.row_count()
    test_size = floor(total_rows / 2)
    train, test = holdout(auto, test_size)

    # step 4
    matrix = knn_eval(train, test, majority_vote, 5, 'mpg', ['cyls', 'weight', 'accl'], [])
    print(matrix)

    # step 5
    avg_acc = (accuracy(matrix, 1) + accuracy(matrix, 2) + accuracy(matrix, 3)) / 3
    print(f' average accuracy is {avg_acc}')

    # step 6
    f = []
    for i in range(3):
        num = 0
        den = 0
        if i+1 in matrix.columns():
            num = (2*recall(matrix, i+1)*precision(matrix, i+1))
            den = (recall(matrix, i+1) + precision(matrix, i+1))
        else:
            fi = 0
        if den != 0:
            fi = num/den
        else:
            fi = 0
        f.append(fi)

    avg_f = (f[0]+f[1]+f[2])/3
    print(f'average f-measure is {avg_f}')


  actual    1    2    3
--------  ---  ---  ---
       1   25   26    0
       2   22   31    0
       3   12   37    0
 average accuracy is 0.5773420479302832
average f-measure is 0.29210472067614923
  actual    1    2    3
--------  ---  ---  ---
       1    0    3   48
       2    0    2   47
       3    0    2   51
 average accuracy is 0.5642701525054467
average f-measure is 0.19466379516630772
  actual    1    2    3
--------  ---  ---  ---
       1    0    8   39
       2    0   16   40
       3    0    7   43
 average accuracy is 0.5904139433551198
average f-measure is 0.289272030651341


Choice 2 of bin size: [< 14, 14-25, > 25]

In [104]:
for i in range(3):
    # step 0
    auto = DataTable(['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
    auto.load('auto-mpg.txt')

    auto = remove_duplicates(auto)
    auto = remove_missing(auto, ['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
    
    # step 1
    discretize(auto, 'mpg', [14.0, 25.0])

    # step 2
    normalize(auto, 'cyls')
    normalize(auto, 'disp')
    normalize(auto, 'hp')
    normalize(auto, 'weight')
    normalize(auto, 'accl')
    normalize(auto, 'year')

    # step 3
    total_rows = auto.row_count()
    test_size = floor(total_rows / 2)
    train, test = holdout(auto, test_size)

    # step 4
    matrix = knn_eval(train, test, majority_vote, 5, 'mpg', ['cyls', 'weight', 'accl'], [])
    print(matrix)

    # step 5
    avg_acc = (accuracy(matrix, 1) + accuracy(matrix, 2) + accuracy(matrix, 3)) / 3
    print(f' average accuracy is {avg_acc}')

    # step 6
    f = []
    for i in range(3):
        num = 0
        den = 0
        if i+1 in matrix.columns():
            num = (2*recall(matrix, i+1)*precision(matrix, i+1))
            den = (recall(matrix, i+1) + precision(matrix, i+1))
        else:
            fi = 0
        if den != 0:
            fi = num/den
        else:
            fi = 0
        f.append(fi)

    avg_f = (f[0]+f[1]+f[2])/3
    print(f'average f-measure is {avg_f}')


  actual    1    2    3
--------  ---  ---  ---
       1    0   18    0
       2    0   83    0
       3    0   52    0
 average accuracy is 0.6949891067538125
average f-measure is 0.2344632768361582
  actual    1    2    3
--------  ---  ---  ---
       1    0   20    0
       2    0   92    0
       3    0   41    0
 average accuracy is 0.7342047930283225
average f-measure is 0.25034013605442174
  actual    1    2    3
--------  ---  ---  ---
       1    0   13    0
       2    0   98    1
       3    0   41    0
 average accuracy is 0.7603485838779956
average f-measure is 0.2602921646746348


Results: The first choice of bins seems to have a better f-measure, but the second choice seems to result in a better accuracy. This is because more things are binned into the second bin for the second choice of bins, which screws the accuracy.

11. Pick a bin size and voting method from above, and redo 6 but with two different k values (i.e., add two more for loops).

k = 3

In [105]:
for i in range(3):
    # step 0
    auto = DataTable(['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
    auto.load('auto-mpg.txt')

    auto = remove_duplicates(auto)
    auto = remove_missing(auto, ['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
    
    # step 1
    discretize(auto, 'mpg', [17.0, 24.0])

    # step 2
    normalize(auto, 'cyls')
    normalize(auto, 'disp')
    normalize(auto, 'hp')
    normalize(auto, 'weight')
    normalize(auto, 'accl')
    normalize(auto, 'year')

    # step 3
    total_rows = auto.row_count()
    test_size = floor(total_rows / 2)
    train, test = holdout(auto, test_size)

    # step 4
    matrix = knn_eval(train, test, majority_vote, 3, 'mpg', ['cyls', 'weight', 'accl'], [])
    print(matrix)

    # step 5
    avg_acc = (accuracy(matrix, 1) + accuracy(matrix, 2) + accuracy(matrix, 3)) / 3
    print(f' average accuracy is {avg_acc}')

    # step 6
    f = []
    for i in range(3):
        num = 0
        den = 0
        if i+1 in matrix.columns():
            num = (2*recall(matrix, i+1)*precision(matrix, i+1))
            den = (recall(matrix, i+1) + precision(matrix, i+1))
        else:
            fi = 0
        if den != 0:
            fi = num/den
        else:
            fi = 0
        f.append(fi)

    avg_f = (f[0]+f[1]+f[2])/3
    print(f'average f-measure is {avg_f}')


  actual    1    2    3
--------  ---  ---  ---
       1   33   14    1
       2   31   23    3
       3   23   24    1
 average accuracy is 0.5816993464052288
average f-measure is 0.30548508214002296
  actual    1    2    3
--------  ---  ---  ---
       1   13    8   24
       2   24    9   23
       3   14   12   26
 average accuracy is 0.5424836601307189
average f-measure is 0.29953267973856207
  actual    1    2    3
--------  ---  ---  ---
       1    0    3   42
       2    0    7   41
       3    0    2   58
 average accuracy is 0.616557734204793
average f-measure is 0.2701492537313433


k = 10

In [106]:
for i in range(3):
    # step 0
    auto = DataTable(['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
    auto.load('auto-mpg.txt')

    auto = remove_duplicates(auto)
    auto = remove_missing(auto, ['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
    
    # step 1
    discretize(auto, 'mpg', [17.0, 24.0])

    # step 2
    normalize(auto, 'cyls')
    normalize(auto, 'disp')
    normalize(auto, 'hp')
    normalize(auto, 'weight')
    normalize(auto, 'accl')
    normalize(auto, 'year')

    # step 3
    total_rows = auto.row_count()
    test_size = floor(total_rows / 2)
    train, test = holdout(auto, test_size)

    # step 4
    matrix = knn_eval(train, test, majority_vote, 10, 'mpg', ['cyls', 'weight', 'accl'], [])
    print(matrix)

    # step 5
    avg_acc = (accuracy(matrix, 1) + accuracy(matrix, 2) + accuracy(matrix, 3)) / 3
    print(f' average accuracy is {avg_acc}')

    # step 6
    f = []
    for i in range(3):
        num = 0
        den = 0
        if i+1 in matrix.columns():
            num = (2*recall(matrix, i+1)*precision(matrix, i+1))
            den = (recall(matrix, i+1) + precision(matrix, i+1))
        else:
            fi = 0
        if den != 0:
            fi = num/den
        else:
            fi = 0
        f.append(fi)

    avg_f = (f[0]+f[1]+f[2])/3
    print(f'average f-measure is {avg_f}')


  actual    1    2    3
--------  ---  ---  ---
       1    4   40    5
       2    0   56    4
       3    0   38    6
 average accuracy is 0.6209150326797386
average f-measure is 0.31055093812125184
  actual    1    2    3
--------  ---  ---  ---
       1    5    1   34
       2    3    1   54
       3    4    0   51
 average accuracy is 0.5816993464052288
average f-measure is 0.2504714071724381
  actual    1    2    3
--------  ---  ---  ---
       1    1    7   32
       2    0    7   56
       3    0    6   44
 average accuracy is 0.5599128540305011
average f-measure is 0.23365722337218076


# 3. Issues, Challenges, and Observations

*TODO: Write down any issues and/or challenges that you had with the assignment. In addition, write down your observations concerning section 2 and brief explanations for the results you obtained.*

I had some confusion about the knn_eval function but I understood better after talking with Bowers. I am still having small syntax errors which make make things more difficult when trying to code. What I've observed is that the function is still only okay at predicting mpg. No matter the conditions, accuracy seems to stay relatively constant (around 0.5) and f-measure stays low (around 0.3). I also realized how important the choice of bins is, because have the distribution of the bins very skewed messes with your accuracy and f-measure. The results I have show that purposefully chosen bins are probably better than equal-width bins, majority vote is probably better than weighted vote, and a higher k is probably better than a lower k. However, none of these results show dramatic differences.